In [4]:
import numpy as np

"""STVM 계산(3차로 점선)"""
import pandas as pd

import os

start_interval = 1800
end_interval = 5400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1,
    "w7" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

folder_path = r"C:\VISSIM_Workspace\network_new\network_version2\network_version2.3\network_version2.3.0\network03_mer"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
mer_file = mer_list[0]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()

# 원시 데이터 기초 가공처리
with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
        lines = file.readlines()
        # 데이터가 시작하는 인덱스 찾기
        data_start_idx = None

        for j, line in enumerate(lines):
            if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                data_start_idx = j
                break

        # 데이터프레임 생성
        if data_start_idx is not None:

            # 컬럼명 추출 및 공백 제거
            columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

            # 데이터 부분 추출 및 가공
            data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
            data = [line.strip().split(";") for line in data_lines if line.strip()]

            # 데이터프레임 생성
            df = pd.DataFrame(data, columns=columns)

            # 컬럼 내부 데이터 정수형 변환
            for col in columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.strip())

            measure_list  = [10010, 20010, 30010]

            filtered_df = df[(df["t(Entry)"] != -1.00) & (df["Measurem."].isin(measure_list))].reset_index(drop=True)
            bins = np.arange(start_interval, end_interval+1, 300)
            labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

            # 구간 컬럼 추가
            filtered_df["TimeGroup"] = pd.cut(filtered_df["t(Entry)"], bins=bins, labels=labels, right=False)

            # 차종별 대수
            filtered_df["type_100"] = (filtered_df["Vehicle type"] == 100).astype(int)
            filtered_df["type_300"] = (filtered_df["Vehicle type"] == 300).astype(int)
            filtered_df["type_630"] = (filtered_df["Vehicle type"] == 630).astype(int)
            filtered_df["type_640"] = (filtered_df["Vehicle type"] == 640).astype(int)
            filtered_df["type_650"] = (filtered_df["Vehicle type"] == 650).astype(int)

            grouped_df = filtered_df.groupby("TimeGroup", as_index=False).agg(
                평균속도 = ("v[km/h]", "mean"),
                차량수 = ("v[km/h]", "count"),
                type_100 = ("type_100", "sum"),
                type_300 = ("type_300", "sum"),
                type_630 = ("type_630", "sum"),
                type_640 = ("type_640", "sum"),
                type_650 = ("type_650", "sum")
            )

            # 중차량 비율
            heavy_weights = (grouped_df["type_630"] + grouped_df["type_640"] + grouped_df["type_650"]) / grouped_df["차량수"] * 100
            grouped_df["중차량비율"] = heavy_weights

            # 진입부 전체 교통량(5분단위)
            total_vehicle = grouped_df["차량수"] * 12
            grouped_df["진입부 교통량"] = total_vehicle

            # 밀도 -> 5분 단위 집계이기 때문에 시간 환산 시 *12
            density = total_vehicle / grouped_df["평균속도"]
            grouped_df["밀도"] = density

            # 속도변화
            grouped_df["속도변화"] = grouped_df["평균속도"].diff()

            # 밀도변화
            grouped_df["밀도변화"] = grouped_df["밀도"].diff()


            # 유입교통량
            grouped_df["유입교통량"] = 50

            # 유출교통량
            grouped_df["유출교통량"] = 20

            # 진출부 유출 정상성
            grouped_df["진출부유출정상성"] = 0.5

result_df["STVM"] = (
    weights["w1"] * grouped_df["속도변화"] +
    weights["w2"] * grouped_df["밀도변화"] +
    weights["w3"] * grouped_df["중차량비율"] +
    weights["w4"] * (grouped_df["진입부 교통량"] / 2200) +
    weights["w5"] * (grouped_df["유입교통량"]/grouped_df["차량수"]) +
    weights["w6"] * (grouped_df["유출교통량"]/grouped_df["차량수"]) +
    weights["w7"] * (1-grouped_df["진출부유출정상성"])
)

# 평균
mean_stvm = result_df["STVM"].mean(skipna=True)

# 표준편차
std_stvm = result_df["STVM"].std(skipna=True)

# Z-Score 계산
result_df["Z-Score"] = (result_df["STVM"] - mean_stvm) / std_stvm



def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * (z - 1.645) / (z_max - 1.645)) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * (z - 1.282) / (1.645 - 1.282)) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * (z - 1.038) / (1.282 - 1.038)) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * (z - 0.842) / (1.038 - 0.842)) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * (z - 0.676) / (0.842 - 0.676)) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * (z - 0.526) / (0.676 - 0.526)) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * (z - 0.387) / (0.526 - 0.387)) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * (z - 0.255) / (0.387 - 0.255)) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * (z + 0.255) / (0.255 + 0.255)) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * (z + 0.387) / (0.255 + 0.387)) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * (z + 0.526) / (0.387 + 0.526)) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * (z + 0.676) / (0.526 + 0.676)) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * (z + 0.842) / (0.676 + 0.842)) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * (z + 1.038) / (0.842 + 1.038)) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * (z + 1.282) / (1.038 + 1.282)) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * (z + 1.645) / (1.282 + 1.645)) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * (z - z_min) / (-1.645 + abs(z_min))) * 0.5)
    else:
        return np.nan


# z_min, z_max 계산
z_min = result_df["Z-Score"].min()
z_max = result_df["Z-Score"].max()

# 환산점수 컬럼 추가 (Z-Score → 100점 척도 변환)
result_df["환산점수(시간대별)"] = result_df["Z-Score"].apply(lambda z: z_to_score(z, z_min, z_max))

display(result_df)



,STVM,Z-Score,환산점수
0,NaN,NaN,NaN
1,8.594223,0.687607,87.674811
2,5.817150,-0.399038,65.347650
3,6.916146,0.030990,71.401911
4,8.166380,0.520196,84.895615
5,7.047075,0.082221,71.653045
6,8.830351,0.780002,89.066299
7,7.579374,0.290505,80.672451
8,3.578169,-1.275133,55.007400
9,11.565967,1.850426,100.000000
